# EDA

In [65]:
FILE = 'data/copper_production_v1.csv'

In [66]:
import pandas as pd

In [67]:
df = pd.read_csv(FILE, index_col=0)

In [68]:
df.sample(3)

,Machine,Shift,Operator,Date,Cable Failures,Cable Failure Downtime,Other Failures,Other Failure Downtime
63,5,A,9,11/17/2020,1,50,0,0
7,16,B,27,11/14/2020,1,60,1,60
35,9,A,18,11/14/2020,1,25,0,0


In [69]:
df.shape

(154, 8)

## Convert Date to datetime format

In [71]:
# either convert to datetime or do this when loading the file

## Check for & remove duplicates

In [76]:
# a duplicate is defined as two or more records with the same values for all columns in the data
number_dupes = len(df[df.duplicated()])
# report the number of duplicates that were removed
print(number_dupes)

5


In [77]:
# # create dupes
# dupes = df.sample(int(len(df)*0.03))
# dupes_ex = dupes.iloc[-1]
# df = df.append(dupes, ignore_index=True).append(dupes_ex, ignore_index=True)
# df = df.sample(frac=1).reset_index()
# df.to_csv('data/copper_production_v1.csv')

In [ ]:
# df[df.duplicated(subset=['Machine','Shift','Operator','Date'], keep=False)] # these are records that should be combined into one observation & the values summed

## Create columns for 'Total Failures' & 'Total Downtime'

In [ ]:
# insert two new columns to calculate the simple sum of failures & downtime for a given record 

## Identify the Machines responsible for 80% of overall 'Total Failures'

# 

# End